In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [12]:
brochure_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
translater_prompt = "You are a translater how translate the given Borchure."


In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [56]:
def stream_brochure(company_name, url):
    brochure = ""
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        brochure += response
        update_display(Markdown(response), display_id=display_handle.display_id)
     return brochure

In [66]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

In [67]:
def get_user_translater_prompt(language, brochure):
    user_prompt = f"You are translate the following brochure into {language}:\n"
    user_prompt += f"Here are the contents of brochure; return same format as the brochure had before Markdown\n"
    user_prompt += brochure
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [68]:
def translate_brochure(company_name, url):
    brochure = create_brochure(company_name, url)
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translater_prompt},
            {"role": "user", "content": get_user_translater_prompt("Deutsch", brochure)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [69]:
translate_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face: Die KI-Community, die die Zukunft gestaltet

Willkommen bei **Hugging Face**, der führenden Plattform, auf der die Machine-Learning-Community an innovativen Modellen, Datensätzen und Anwendungen zusammenarbeitet. Unsere Mission ist es, die Entwicklung von KI-Technologien zu beschleunigen und gleichzeitig ein inklusives Umfeld für Lernen und Wachstum zu schaffen.

## Unternehmensübersicht

Bei Hugging Face bringen wir Datenwissenschaftler, Ingenieure und KI-Enthusiasten aus der ganzen Welt zusammen, um hochmoderne Machine-Learning-Lösungen zu entwickeln. Unsere Plattform beherbergt über **1 Million Modelle** und **250.000 Datensätze**, was sie zu einer wertvollen Ressource für Praktiker und Forscher macht.

### Hauptangebote:
- **Modelle**: Entdecken Sie eine Vielzahl von angesagten ML-Modellen, darunter Text-, Bild-, Video-, Audio- und 3D-Modelle.
- **Datensätze**: Greifen Sie auf verschiedene Datensätze zu und teilen Sie diese, die auf jede ML-Aufgabe zugeschnitten sind.
- **Spaces**: Arbeiten Sie mit anderen an der Entwicklung und dem Betrieb von Anwendungen, die fortschrittliche KI-Technologien nutzen.
- **Enterprise-Lösungen**: Bieten Sie Teams fortschrittliche Werkzeuge zur Entwicklung von KI mit Unternehmenssicherheit und dedizierter Unterstützung.

## Kultur und Gemeinschaft

Hugging Face ist mehr als nur ein Unternehmen; es ist eine florierende Gemeinschaft von KI-Profis und Lernenden. Unsere Kultur fördert Zusammenarbeit, Experimentierfreude und Teilen. Wir glauben an die Kraft von **Open Source** und tragen aktiv zur Zukunft des Machine Learning durch gemeinschaftliche Werkzeuge wie Transformers und Diffusers bei.

### Gemeinschaftsinitiativen:
- Nehmen Sie an Foren teil und tragen Sie zu Projekten bei.
- Besuchen Sie von der Gemeinschaft geleitete Veranstaltungen, um Wissen und Erkenntnisse auszutauschen.
- Beteiligen Sie sich an Diskussionen auf **GitHub**, **Discord** und anderen sozialen Plattformen.

## Kunden

Mehr als **50.000 Organisationen** nutzen Hugging Face, um ihre KI-Fähigkeiten zu verbessern, darunter Branchenführer wie Amazon, Google, Intel und Microsoft. Unsere Plattform wird sowohl von Unternehmen als auch von gemeinnützigen Organisationen vertraut, die alle bestrebt sind, die Grenzen der KI-Technologie zu erweitern.

## Treten Sie unserem Team bei

Bei Hugging Face suchen wir ständig nach leidenschaftlichen Menschen, die unserem vielfältigen Team beitreten möchten. Wenn Sie sich für KI begeistern und zu bahnbrechenden Projekten beitragen möchten, schauen Sie sich unsere [Karriereseite](https://huggingface.co/jobs) für aktuelle Stellenangebote an!

### Zusammenarbeit mit uns:
- **Innovatives Umfeld**: Arbeiten Sie mit einigen der hellsten Köpfe im Bereich KI zusammen.
- **Wachstumschancen**: Kontinuierliches Lernen und berufliche Entwicklung werden gefördert.
- **Inklusive Kultur**: Wir heißen unterschiedliche Perspektiven und Ideen willkommen und fördern ein Gefühl der Zugehörigkeit für alle Mitarbeiter.

---

Kommen Sie mit uns zusammen, um die Zukunft der künstlichen Intelligenz zu gestalten!
Erfahren Sie mehr unter [huggingface.co](https://huggingface.co)
